In [ ]:
from collections import defaultdict

# 削除可能ヒープ
# self.heap: ヒープ
# self.delete_heap: 削除予約待ちのヒープ
# self.item_cnt: defaultdict(key:heapの要素からdelete_heapを除いた要素，value:その要素が何個あるか)
# len(): ヒープから削除予約待ちのヒープを除いた要素数
# __repr__(): ヒープから削除予約待ちのヒープを除いた要素をリストにして返す
# __iter__(): ヒープから削除予約待ちのヒープを除いた要素をイテレータとして返す
# pop(): ヒープの最小値を取り出す
# push(item): ヒープにitemを追加する
# remove(item): ヒープからitemを削除する(内部的には削除予約しておいて，その要素がヒープの先頭にきたら削除する)
# get_item_cnt(item): ヒープ内のitemの個数を返す
from heapq import *
class DeletableHeap:
    def __init__(self,heap):
        self.heap = heap
        self.delete_heap = []
        self.item_cnt = defaultdict(int)
        for item in heap:
            self.item_cnt[item] += 1
    def __len__(self):
        return len(self.heap) - len(self.delete_heap)
    def __repr__(self):
        return str(list(self))
    def __iter__(self):
        j = 0
        for item in self.heap:
            if j < len(self.delete_heap):
                if item == self.delete_heap[j]:
                    j += 1
                    continue
            yield item
    def pop(self):
        while self.heap and self.delete_heap:
            if self.heap[0] == self.delete_heap[0]:
                heappop(self.heap)
                heappop(self.delete_heap)
            else:
                break
        return_item = heappop(self.heap)
        self.item_cnt[return_item] -= 1
        return return_item
    def push(self,item):
        heappush(self.heap,item)
        self.item_cnt[item] += 1
    def remove(self,item):
        heappush(self.delete_heap,item)
        self.item_cnt[item] -= 1
        while self.heap and self.delete_heap:
            if self.heap[0] == self.delete_heap[0]:
                heappop(self.heap)
                heappop(self.delete_heap)
            else:
                break
    def get_item_cnt(self,item):
        return self.item_cnt[item]

# 配列から完全になくなった時だけheapに追加する
# 配列にないものが出てきたらheapから削除する
def main():
    N,Q = map(int,input().split())
    A = list(map(int,input().split()))
    cnt_dict = defaultdict(int)
    mex = DeletableHeap([i for i in range(N+1)])
    for a in A:
        cnt_dict[a] += 1
        if cnt_dict[a] == 1:
            mex.remove(a)
    for _ in [0]*Q:
        i,x = map(int,input().split())
        i -= 1
        cnt_dict[A[i]] -= 1
        if cnt_dict[A[i]] == 0:
            mex.push(A[i])
        A[i] = x
        cnt_dict[A[i]] += 1
        if cnt_dict[A[i]] == 1:
            mex.remove(A[i])
        print(mex.heap[0])

if __name__ == "__main__":
    main()

In [ ]:
# セグメント木
# op: op(x,y) return x*y みたいな演算(関数)
# e: 単位元を返す関数，op(x,e()) = op(e(),x) = x になるようなe
# n: 要素数
# v: 要素の配列
class SegTree:
    def __init__(self, op, e, n, v=None):
        # 要素数
        self._n = n
        # 二項演算
        self._op = op
        # 単位元を返す関数
        self._e = e
        # セグメント木の深さ(根は0)
        self._log = (n - 1).bit_length()
        # 葉の数
        self._size = 1 << self._log
        # セグメント木(0番目の要素は使わない，1番目の要素が根)
        self._d = [self._e()] * (self._size << 1)
        if v is not None:
            for i in range(self._n):
                self._d[self._size + i] = v[i]
            for i in range(self._size - 1, 0, -1):
                self._d[i] = self._op(self._d[i << 1], self._d[i << 1 | 1])
    # p番目の要素をxに変更(pは0-indexed)
    def set(self, p, x):
        p += self._size
        self._d[p] = x
        while p:
            self._d[p >> 1] = self._op(self._d[p], self._d[p ^ 1])
            p >>= 1
    # p番目の要素にxを加算(pは0-indexed)
    def add(self, p, x):
        p += self._size
        self._d[p] += x
        while p:
            self._d[p >> 1] = self._op(self._d[p], self._d[p ^ 1])
            p >>= 1
    # p番目の要素を返す(pは0-indexed)
    def get(self, p):
        return self._d[p + self._size]
    # op(d[l], d[l+1], ..., d[r-1])を返す(l,rは0-indexedの半開区間)
    def prod(self, l, r):
        sml, smr = self._e(), self._e()
        l += self._size
        r += self._size
        while l < r:
            if l & 1:
                sml = self._op(sml, self._d[l])
                l += 1
            if r & 1:
                r -= 1
                smr = self._op(self._d[r], smr)
            l >>= 1
            r >>= 1
        return self._op(sml, smr)
    # op(d[0], d[1], ..., d[n-1])を返す
    def all_prod(self):
        return self._d[1]
    # op(d[0], d[1], ..., d[n-1]) = d[i] となる最も左のiを返す(min, maxで使う)
    def minmax_left(self):
        minmax = self._d[1]
        idx = 1
        while idx < self._size:
            if self._d[idx*2] == minmax:
                idx *= 2
            else:
                idx = idx*2+1
        return idx-self._size
    
def main():
    N,Q = map(int,input().split())
    A = list(map(int,input().split()))
    seg = SegTree(min, lambda: 10**6, N+1)
    for a in A:
        if a < N+1:
            seg.add(a,1)
    for _ in [0]*Q:
        i,x = map(int,input().split())
        i -= 1
        if A[i] < N+1:
            seg.add(A[i],-1)
        A[i] = x
        if A[i] < N+1:
            seg.add(A[i],1)
        print(seg.minmax_left())

if __name__ == "__main__":
    main()